In [1]:
from collections import defaultdict
import imblearn

In [2]:
import pandas as pd
import nltk
import sklearn as sk
import gensim.models
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ast import literal_eval
import predictor
sns.set()

clean = pd.read_csv("/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/CleanData(v. analysis).csv")

colname = ['id', 'condition', 'wish', 'outcome', 'outcome2', 'useful', 'mins', 'other', 'reuse', 'clarity', 'benefit', 'interest', 'learn']
clean = clean.iloc[:256,:13]
clean.columns= colname
clean.fillna("", inplace=True)

In [3]:
clean['all'] = clean.wish + " " + clean.outcome + " " + clean.outcome2
clean.drop(columns = ['wish', 'outcome', 'outcome2'], inplace=True)

In [4]:
coders = pd.read_csv('/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/MCII coding replication - coding file(v. analysis).csv')
colname = ['id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 's1', 's2', 's3', 's4', 's5', 's6']
coders = coders.iloc[:256,:13]
coders.columns= colname
coders.fillna(1, inplace=True)
coders['d'] = round(coders.loc[:,'d1':'d6'].mean(axis=1))
coders['s'] = round(coders.loc[:,'s1':'s6'].mean(axis=1))
clean = pd.concat([clean, coders], join='outer', axis=1)
clean.head()

,id,condition,useful,mins,other,reuse,clarity,benefit,interest,learn,...,d5,d6,s1,s2,s3,s4,s5,s6,d,s
0,558af269fdf99b7dfef4d7bd,MC,7.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,...,2.0,5.0,2.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0
1,55b1add5fdf99b6731f04c5a,MC,1.0,1.0,2.0,1.0,3.0,1.0,2.0,2.0,...,5.0,5.0,4.0,3.0,5.0,4.0,3.0,5.0,5.0,4.0
2,5626a6cfddff3c0011bcd3d8,I,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,3.0,5.0,4.0,3.0,5.0,4.0,4.0,4.0
3,56a5da6181a8ca000d3e7f40,MC,4.0,1.0,4.0,3.0,2.0,3.0,5.0,4.0,...,5.0,5.0,3.0,3.0,4.0,3.0,3.0,4.0,4.0,3.0
4,56df2ea2452e1300056bc7df,MC,3.0,1.0,4.0,4.0,3.0,3.0,4.0,3.0,...,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0


In [9]:
# Trying to see the difference between unbalanced and balanced training for directness coding

mypredictor = predictor.predictor(clean, 'all')
mypredictor.train_on('d')
mypredictor.test()

{'fit_time': array([0.24190497, 0.22401786, 0.27497911]), 'score_time': array([0.00149488, 0.00083899, 0.00057173]), 'test_score': array([0.45454545, 0.54545455, 0.39473684]), 'train_score': array([0.86928105, 0.82352941, 0.81818182])}
0.6923076923076923


0.6923076923076923

In [10]:
balancer = imblearn.over_sampling.SMOTE()
X, y = balancer.fit_resample(mypredictor.get_X(), clean['d'])

In [11]:
mypredictor.manual_X_y(X,y)
mypredictor.test()

{'fit_time': array([0.71485496, 0.7453289 , 0.84247804]), 'score_time': array([0.0007112 , 0.00162101, 0.00092387]), 'test_score': array([0.75      , 0.73469388, 0.73469388]), 'train_score': array([0.93877551, 0.94915254, 0.9220339 ])}
0.8461538461538461


0.8461538461538461

In [12]:
# Trying to see the difference between unbalanced and balanced training for specificity coding

mypredictor = predictor.predictor(clean, 'all')
mypredictor.train_on('s')
mypredictor.test()

{'fit_time': array([0.643507  , 0.78533506, 0.42483783]), 'score_time': array([0.00369596, 0.00069571, 0.0005939 ]), 'test_score': array([0.66233766, 0.55844156, 0.53947368]), 'train_score': array([0.75163399, 0.78431373, 0.77922078])}
0.6538461538461539


0.6538461538461539

In [15]:
balancer = imblearn.over_sampling.SMOTE(k_neighbors = 3)
X, y = balancer.fit_resample(mypredictor.get_X(), clean['s'])
mypredictor.manual_X_y(X,y)
mypredictor.test()

{'fit_time': array([0.51963806, 0.99074697, 1.08027983]), 'score_time': array([0.00114989, 0.00148106, 0.0006299 ]), 'test_score': array([0.78034682, 0.83236994, 0.81395349]), 'train_score': array([0.91594203, 0.93913043, 0.95086705])}
0.8913043478260869


0.8913043478260869